In [46]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statistics
from enum import Enum
import importlib

import plotly.express as px
import plotly.graph_objects as go

from customtypes import POSITIONS

In [39]:
from dynastyrequests import *
players_response = request_players()
rosters_response = request_rosters()
projections_response = request_projections()
ktc_response = request_ktc()
fantasy_calc_response = request_fantasy_calc()

Linking to keeptradecut.com's SF rankings...: 100%|██████████| 10/10 [00:05<00:00,  1.79page/s]


In [40]:
players = get_players(players_response)
rosters = get_rosters(rosters_response)
projections = get_projections(projections_response)
ktc = get_ktc_values(ktc_response)
fantasy_calc = get_fantasy_calc_values(fantasy_calc_response)

## BUILD LEAGUE

In [56]:
imported_module = importlib.import_module("teams")
importlib.reload(imported_module)
from teams import *
from teams import Team, build_league, set_team_starters
league = build_league(rosters,players,projections,ktc,fantasy_calc)
for team in league:
   set_team_starters(team)

## Create median/average data

In [66]:
imported_module = importlib.import_module("leaguestats")
importlib.reload(imported_module)
from leaguestats import *
league_stats = LeagueStats()
league_wide_statistics = {
    'average_starter_ppg': 0,
    'median_starter_ppg': 0,
    'average_starters': {},
    'median_starters': {},
    'average_best_bench': {},
    'median_best_bench': {}
}
calculate_league_starter_stats(league, league_wide_statistics)
calculate_league_bench_stats(league, league_wide_statistics)

In [65]:
league[8].starting_roster['QB']

,position,age,Player,FPTS,KTC_Value,FantasyCalc_Value,redraftValue,redraftDynastyValueDifference,redraftDynastyValuePercDifference,FPPG,Starter
0,QB,30,dak prescott,357.41,5617.0,4599.0,6934.0,2335.0,50.0,21.02,False


In [12]:
{'average_starter_ppg': 103.98,
 'median_starter_ppg': 104.58500000000001,
 'average_starters': {'QB': [20.41357142857143],
  'WR': [12.552857142857144, 11.024285714285712],
  'RB': [12.355],
  'TE': [10.139999999999999],
  'FLEX': [11.04857142857143, 10.172857142857142],
  'SFLEX': [16.27285714285714]},
 'median_starters': {'QB': [20.200000000000003],
  'WR': [12.36, 11.3],
  'RB': [12.605],
  'TE': [10.52],
  'FLEX': [10.55, 10.105],
  'SFLEX': [16.83]},
 'average_best_bench': {'WR': [8.492857142857142, 7.115],
  'RB': [7.040714285714287, 6.115384615384615],
  'TE': [7.137142857142857],
  'FLEX': [8.960714285714285, 8.290714285714285, 7.394285714285714],
  'SFLEX': [11.905]},
 'median_best_bench': {'WR': [8.3, 6.71],
  'RB': [6.385, 6.04],
  'TE': [7.07],
  'FLEX': [8.585, 8.005, 6.945],
  'SFLEX': [10.475000000000001]}}


{'average_starter_ppg': 103.98,
 'median_starter_ppg': 104.58500000000001,
 'average_starters': {'QB': [20.41357142857143],
  'WR': [12.552857142857144, 11.024285714285712],
  'RB': [12.355],
  'TE': [10.139999999999999],
  'FLEX': [11.04857142857143, 10.172857142857142],
  'SFLEX': [16.27285714285714]},
 'median_starters': {'QB': [20.200000000000003],
  'WR': [12.36, 11.3],
  'RB': [12.605],
  'TE': [10.52],
  'FLEX': [10.55, 10.105],
  'SFLEX': [16.83]},
 'average_best_bench': {'WR': [8.492857142857142, 7.115],
  'RB': [7.040714285714287, 6.115384615384615],
  'TE': [7.137142857142857],
  'FLEX': [8.960714285714285, 8.290714285714285, 7.394285714285714],
  'SFLEX': [11.905]},
 'median_best_bench': {'WR': [8.3, 6.71],
  'RB': [6.385, 6.04],
  'TE': [7.07],
  'FLEX': [8.585, 8.005, 6.945],
  'SFLEX': [10.475000000000001]}}

## Calculate team scores

In [13]:
def normalize_score(score):
    return (score * 100)

# 1, 0.5, 0.25 ...
def get_bench_weights(count):
    return [1 / pow(2,i) for i in range(count)]

In [23]:
def calculate_overall_starter_score_for_team(team):
    # sum teams starter ppg             # get the percentage of the league average and normalize
    starter_score_vs_avg = team.get_starters()['FPPG'].sum() / league_wide_statistics['average_starter_ppg']
    starter_score_vs_avg = normalize_score(starter_score_vs_avg)
    starter_score_vs_median = team.get_starters()['FPPG'].sum() / league_wide_statistics['median_starter_ppg']
    starter_score_vs_median = normalize_score(starter_score_vs_median)
    team.scores['starter_score'] = np.average([starter_score_vs_avg, starter_score_vs_median])

In [28]:
for team in league:
    calculate_overall_starter_score_for_team(team)

In [15]:
# (team: Team, pos: 'Position', starter: boolean, number_of_players_to_score: int)
def calculate_score_for_position(team: Team, position, starter: bool, number_of_players_to_score):
    if starter:
        players = team.get_position_starters(position, count=number_of_players_to_score)['FPPG']
        avgs = league_wide_statistics['average_starters'][position.name]
        medians = league_wide_statistics['median_starters'][position.name]
    else:
        players = team.get_position_backups(position, count=number_of_players_to_score)['FPPG']
        avgs = league_wide_statistics['average_best_bench'][position.name]
        medians = league_wide_statistics['median_best_bench'][position.name]

    # will be a list of ith backup / ith league average backup
    # [ best_backup / league_best_backup, second_backup / league_second_backup, ...]
    vs_avg = np.array(players) / np.array(avgs)
    vs_avg = normalize_score(vs_avg)
    vs_median = np.array(players) / np.array(medians)
    vs_median = normalize_score(vs_median)

    # take an element-wise mean of vs_avg and medians
    # [ mean(best_backup / league_best_avg, best_backup / league_best_median), ...]
    combined = [np.mean([avg, median]) for avg, median in zip(vs_avg, vs_median)]
    
    # take the weighted average of the scores and insert single numeric score
    if starter:
        team.scores['starters'][position.name] = np.average(combined, weights=get_bench_weights(number_of_players_to_score))
    else:
        team.scores['bench'][position.name] = np.average(combined, weights=get_bench_weights(number_of_players_to_score))

def calculate_bench_score_for_team(team):
    for position in POSITIONS:
        if position != POSITIONS.QB: # hacky, but temporary fix for there being no backup "QB" (sflex instead)
            calculate_score_for_position(team, position, False, position.bench_count())

def calculate_starter_score_for_team(team):
    for position in POSITIONS:
        calculate_score_for_position(team, position, True, position.starter_count())

In [16]:
for team in league:
    calculate_bench_score_for_team(team)
    calculate_starter_score_for_team(team)

In [17]:
league[8].scores

{'starter_score': 110.84017078028384,
 'starters': {'QB': 103.51505935085618,
  'WR': 102.19207485961654,
  'RB': 122.59827992855804,
  'TE': 109.62082180274635,
  'FLEX': 115.17369011164072,
  'SFLEX': 117.72711702377538},
 'bench': {'WR': 143.8344633543636,
  'RB': 164.44562602421058,
  'TE': 75.0775414158741,
  'FLEX': 137.58200145422808,
  'SFLEX': 169.4137458539612}}

# what makes a contender?
# Starters, and their projected points

# all scores are on a scale of 100
  # starter score - weight 50%
    # percentage above and below the league median ppg, normalized around 50

  # Flex depth score - weight 25%
    # top bench asset, percentage against league median best bench asset
    # second bench asset, ...
    # third bench asset, ...
    # bench assets are combined into one depth score, with decreasing weights per rank
      # 1st bench spot - 50%
      # 2nd bench spot - 35%
      # 3rd bench post - 15%

  # RB depth score
    # similar to flex depth, but two assets - rb only
  # WR depth score
    # similar to flex depth, but two assets - wr only
  # TE depth score
    # similar to flex depth, but one asset - te only

  # sflex depth - weight 25%
    # third superflex player, percentage against league median third superflex player

# take combined / weighted scores into a final

# consider adding overall position group scores too
# weighted combination of 1st, 2nd, nth best QB


# coming up with weights. use historical injury data to come up with percentages for how often players of a given percentage get injured

# How to visualize a trade

# create a copy of the team
# insert newly aqured players
# delete moved players
# recalculate roster_starters
# recalculate contender data
# revisualize whatever